**IMPORT STATEMENTS**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Input, GRU, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import cv2
import pathlib
import glob
import shutil
import os
import random
from keras.callbacks import EarlyStopping
from tensorflow.python.keras.layers import Layer
from tensorflow.python.keras import backend as K

In [ ]:
#--------------------------------caution: terminal commands ---------------------------------------------
%cd
%cd .keras/datasets/
!rm -r *

/root
[Errno 2] No such file or directory: '.keras/datasets/'
/root
rm: cannot remove '*': No such file or directory


In [ ]:
########################################### download data from given url ###############################################

dataset_url = "https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar"
data_dir = tf.keras.utils.get_file('dakshina_dataset_v1.0', origin=dataset_url, untar=True)

2008342528/2008340480 [==============================] - 18s 0us/step


In [ ]:
#----------------------------------terminal command -----------------------------------------------
%cd /root/.keras/datasets/dakshina_dataset_v1.0/hi/lexicons/
!ls

/root/.keras/datasets/dakshina_dataset_v1.0/hi/lexicons
hi.translit.sampled.dev.tsv   hi.translit.sampled.train.tsv
hi.translit.sampled.test.tsv


In [ ]:
train_data_path = "hi.translit.sampled.train.tsv"
test_data_path = "hi.translit.sampled.test.tsv"
validation_data_path = "hi.translit.sampled.dev.tsv"

**UTILITY FUNCTION FOR DATA PRE-PROCESSING**

In [ ]:
################################################ preparing the data in required format #################################################

def data(path,input_token_index,target_token_index):
  input_texts = []
  target_texts = []
  with open(path, "r", encoding="utf-8") as f:
      lines = f.read().split("\n")
  del lines[-1]
  for line in lines:
      target_text, input_text, _ = line.split("\t")

      input_text = "\t"+input_text + "\n"
      target_text = "\t"+target_text + "\n"

      input_texts.append(input_text)
      target_texts.append(target_text)

  encoder_input_data = np.zeros( (len(input_texts), max_encoder_seq_length), dtype="int32")
  decoder_input_data = np.zeros( (len(input_texts), max_decoder_seq_length), dtype="int32")
  decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="int32")

  for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
      for t, char in enumerate(input_text):
          encoder_input_data[i, t] = input_token_index[char]
      encoder_input_data[i, t + 1 :] = input_token_index[" "]
      for t, char in enumerate(target_text):
          # decoder_target_data is ahead of decoder_input_data by one timestep
          decoder_input_data[i, t] = target_token_index[char]
          if t > 0:
              # decoder_target_data will be ahead by one timestep
              # and will not include the start character.
              decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
      decoder_input_data[i, t + 1 :] = target_token_index[" "]
      decoder_target_data[i, t:, target_token_index[" "]] = 1.0
    
  return (encoder_input_data, decoder_input_data, decoder_target_data, target_texts, input_texts)

**PREPROCESSING THE DATA**

In [ ]:
################################################ preprocessing the train data and getting dictionaries #################################################

input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(train_data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")

del lines[-1]
for line in lines:
    target_text, input_text, _ = line.split("\t")
    input_text = "\t"+input_text + "\n"
    target_text = "\t"+target_text + "\n"

    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters.add(" ")
target_characters.add(" ")

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])


Number of samples: 44204
Number of unique input tokens: 29
Number of unique output tokens: 66
Max sequence length for inputs: 22
Max sequence length for outputs: 21


**CUSTOM ATTENTION LAYER**

In [ ]:
class AttentionLayer(Layer):

    ############################################# constructor for class AttentionLayer ##########################################
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    #########################################  Function to build ########################################################
    def build(self, input_shape):
        assert isinstance(input_shape, list)

        #-------------------------------------- Create a trainable weight variable for this layer-------------------------------------- 

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape) 


    #######################################  Function to call #################################
    def call(self, inputs, verbose=False):

        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        ##############################  Function for Energy Step ##############################
        def energy_step(inputs, states):

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            #------------------Some parameters required for shaping tensors----------------
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            #------------------Computing S.Wa where S=[s0, s1, ..., si]--------------------
            W_a_dot_s = K.dot(encoder_out_seq, self.W_a)

            #------------------Computing hj.Ua---------------------------------------------
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)
            if verbose:
                print('Ua.h>', U_a_dot_h.shape)

            #------------------Computing tanh(S.Wa + hj.Ua)--------------------------------
            Ws_plus_Uh = K.tanh(W_a_dot_s + U_a_dot_h)
            if verbose:
                print('Ws+Uh>', Ws_plus_Uh.shape)

            #------------------Computing softmax(va.tanh(S.Wa + hj.Ua))-------------------------------
            e_i = K.squeeze(K.dot(Ws_plus_Uh, self.V_a), axis=-1)
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)

            return e_i, [e_i]

        ####################### Step function for computing ci using ei ###############################
        def context_step(inputs, states):            

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]

        fake_state_c = K.sum(encoder_out_seq, axis=1)
        fake_state_e = K.sum(encoder_out_seq, axis=2)  

        #-----------------------Computing energy outputs -----------------------------------------------
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )

        #-----------------------Computing context vectors ----------------------------------------------
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )

        return c_outputs, e_outputs

    ############################ Outputs produced by the layer ##########################################
    def compute_output_shape(self, input_shape):        
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]


**MACHINE TRANSLITERATOR**


In [ ]:

class Machine_Transliterator():

  ############################################# constructor for class Machine_Transliterator ##########################################

  def __init__(self,max_encoder_seq_length,max_decoder_seq_length,encoder_embed_size, decoder_embed_size,
               num_encoder_layers,num_decoder_layers,epochs, hidden_layer_size,
               num_encoder_tokens, cell_type, num_decoder_tokens,input_token_index, target_token_index, 
               activation="softmax",optimizer="rmsprop",dropout=0.05):
    
    self.cell_type= cell_type
    self.hidden_layer_size = hidden_layer_size  
    self.optimizer = optimizer
    self.activation = activation   
    self.dropout=dropout

    #-------------------------------------- Number of hidden layers -------------------------------------

    self.num_encoder_layers = num_encoder_layers
    self.num_decoder_layers=num_decoder_layers

    #-------------------------------------- sequence length -------------------------------------
    self.max_decoder_seq_length=max_decoder_seq_length
    self.max_encoder_seq_length=max_encoder_seq_length

    #---------------------------------------------Embedding size-------------------------------------
    self.encoder_embed_size = encoder_embed_size
    self.decoder_embed_size = decoder_embed_size
    
    #-----------------information obtained after preprocessing of data-------------------------------------
    self.num_encoder_tokens = num_encoder_tokens
    self.num_decoder_tokens = num_decoder_tokens

    #-----------------------------dictionaries----------------------------------------------------
    self.input_token_index = input_token_index
    self.target_token_index = target_token_index

 


#########################################function to build model ###########################################

  def build_model(self):
    
    encoder_inputs = keras.Input(shape=(None,))  
    encoder_embedding_output = tf.keras.layers.Embedding(input_dim = self.num_encoder_tokens, output_dim = self.encoder_embed_size, name = "encoder_embedding_layer")(encoder_inputs)

#------------------------------ if cell type = LSTM -------------------------------------------------------------
    if self.cell_type == "lstm":
     #--------------------- encoder -----------------------------------
      encoder = keras.layers.LSTM(self.hidden_layer_size, return_state=True,return_sequences=True,dropout=self.dropout,use_bias=True, name = "encoder_layer_0")
      encoder_outputs, state_h, state_c = encoder(encoder_embedding_output)
      encoder_states = [state_h, state_c]

      #---------------------------decoder ---------------------------------------------------
      decoder_inputs = keras.Input(shape=(None, )) 
      decoder_embedding_output = tf.keras.layers.Embedding(input_dim = self.num_decoder_tokens, output_dim = self.decoder_embed_size, name = "decoder_embedding_layer")(decoder_inputs)

      decoder = keras.layers.LSTM(self.hidden_layer_size, return_sequences=True, return_state=True,dropout=self.dropout,use_bias=True, name = "decoder_layer_0")
      decoder_outputs, _, _= decoder(decoder_embedding_output, initial_state = encoder_states)

      attn_layer = AttentionLayer(name='attention_layer')
      attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])
      decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

      


#------------------------------ if cell type = Simple RNN -------------------------------------------------------------
    elif self.cell_type == "rnn":
      #--------------------- encoder -----------------------------------
      encoder_inputs = keras.Input(shape=(self.max_encoder_seq_length,))
      encoder_embedding_output = tf.keras.layers.Embedding(input_dim = self.num_encoder_tokens, output_dim = self.encoder_embed_size, name = "encoder_embedding_layer", input_length=self.max_encoder_seq_length)(encoder_inputs)

      encoder = keras.layers.SimpleRNN(self.hidden_layer_size, return_state=True,return_sequences=True,dropout=self.dropout, use_bias=True, name = "encoder_layer_0", unroll=True)
      encoder_outputs, state = encoder(encoder_embedding_output)
      encoder_states = [state]

      #---------------------------decoder ---------------------------------------------------
      decoder_inputs = keras.Input(shape=(self.max_decoder_seq_length, ))
      decoder_embedding_output = tf.keras.layers.Embedding(input_dim = self.num_decoder_tokens, output_dim = self.decoder_embed_size, name = "decoder_embedding_layer", input_length=self.max_decoder_seq_length)(decoder_inputs)

      decoder = keras.layers.SimpleRNN(self.hidden_layer_size, return_state=True,return_sequences=True, dropout=self.dropout, use_bias=True, name = "decoder_layer_0", unroll=True)
      decoder_outputs, _ = decoder(decoder_embedding_output, initial_state = encoder_states)

      attn_layer = AttentionLayer(name='attention_layer')
      attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])
      decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])



#------------------------------ if cell type = GRU -------------------------------------------------------------
    elif self.cell_type == "gru":
      #--------------------- encoder -----------------------------------
      encoder = keras.layers.GRU(self.hidden_layer_size, return_state=True,return_sequences=True,dropout=self.dropout,use_bias=True, name = "encoder_layer_0")
      encoder_outputs, state = encoder(encoder_embedding_output)
      encoder_states = [state]

      #---------------------------decoder ---------------------------------------------------
      decoder_inputs = keras.Input(shape=(None, ))      
      decoder_embedding_output = tf.keras.layers.Embedding(input_dim = self.num_decoder_tokens, output_dim = self.decoder_embed_size, name = "decoder_embedding_layer")(decoder_inputs)
      
      decoder = keras.layers.GRU(self.hidden_layer_size, return_state=True,return_sequences=True, dropout=self.dropout,use_bias=True, name = "decoder_layer_0")
      decoder_outputs, _ = decoder(decoder_embedding_output, initial_state = encoder_states)

      attn_layer = AttentionLayer(name='attention_layer')
      attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])
      decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

    decoder_dense = keras.layers.Dense(self.num_decoder_tokens, activation = self.activation,use_bias=True, name = "dense")
    dense_time = TimeDistributed(decoder_dense, name='time_distributed_layer')
    decoder_pred = dense_time(decoder_concat_input)

    model = keras.Model([encoder_inputs, decoder_inputs], decoder_pred)
    
    #-----------------compile the model -------------------------------------
    model.compile(
         optimizer=self.optimizer,
         loss="categorical_crossentropy",
         metrics=["accuracy"]
         ) 

#-------------------------- return final model ---------------------------------------------------------
    return model

#########################################function for training the model ###########################################

  def train_model(self,encoder_input_data,decoder_input_data,decoder_target_data,epochs,batch_size,
                  val_encoder_input_data, val_decoder_input_data, val_decoder_target_data):
    
     model=self.build_model()
  
     model.fit(
        [encoder_input_data, decoder_input_data],
        decoder_target_data,
        batch_size = batch_size,
        epochs = epochs,
        validation_data = ([val_encoder_input_data, val_decoder_input_data],val_decoder_target_data),
        callbacks=[EarlyStopping(patience = 5)]

        # callbacks=[WandbCallback(), EarlyStopping(patience = 5)]
        )
     return model

######################################### function for inference on the model ###########################################
  
  def sampling_inference(self, model, num_encoder_layers, num_decoder_layers):

    encoder_model = None
    decoder_model = None
    if self.cell_type == "lstm":

      #-----------------ENCODER----------------------------
      e_inputs = model.input[0]
      e_embed_layer = model.get_layer("encoder_embedding_layer")
      encoder_inputs = e_embed_layer(e_inputs)

      encoder_cell = model.get_layer("encoder_layer_" + str(num_encoder_layers-1))
      encoder_outputs, state_h_enc, state_c_enc = encoder_cell(encoder_inputs)
      encoder_states = [state_h_enc, state_c_enc]

      #----------------encoder model-------------------------------
      encoder_model = keras.Model(inputs = e_inputs, outputs = [encoder_outputs, state_h_enc, state_c_enc])

      #------------------DECODER------------------------------
      decoder_state_input_h = keras.Input(shape=(self.hidden_layer_size,), name="input_decoder_h")
      decoder_state_input_c = keras.Input(shape=(self.hidden_layer_size,), name="input_decoder_c")
      decoder_hidden_state_input = Input(shape=(22,self.hidden_layer_size))
      decoder_state_inputs = [decoder_state_input_h, decoder_state_input_c]

      d_inputs = model.input[1] #input_2
      d_embed_layer = model.get_layer("decoder_embedding_layer")
      decoder_inputs = d_embed_layer(d_inputs)

      #---------------decoder cell layers---------------------------
      decoder_cell = model.get_layer("decoder_layer_" + str(num_decoder_layers-1))
      decoder_outputs, state_h_dec, state_c_dec = decoder_cell(decoder_inputs, initial_state = decoder_state_inputs)
      decoder_states = [state_h_dec, state_c_dec]

      #-------------------------attention inference ------------------
      attn_layer = model.get_layer("attention_layer")
      attn_inf_out, attn_inf_states = attn_layer([decoder_hidden_state_input, decoder_outputs])

      #------------------------concatenation----------------------------
      concate = model.get_layer("concat_layer")
      decoder_inf_concat = concate([decoder_outputs, attn_inf_out])

      #-------A dense softmax layer to get generate probability over the target vocabulary --------------------
      time_dist_dense = model.get_layer("time_distributed_layer")
      decoder_inf_pred = time_dist_dense(decoder_inf_concat)


      decoder_model = keras.Model( 
                              [d_inputs] + [decoder_hidden_state_input, decoder_state_input_h, decoder_state_input_c],
                              [decoder_inf_pred] + [state_h_dec, state_c_dec]
                            )
    #-------------------------------------if cell type = Simple RNN or GRU---------------------------------------------
    elif self.cell_type =="rnn" or self.cell_type =="gru":

      #---------------------------ENCODER---------------------------------
      e_inputs = model.input[0]
      e_embed_layer = model.get_layer("encoder_embedding_layer")
      encoder_inputs = e_embed_layer(e_inputs)

      encoder_cell = model.get_layer("encoder_layer_" + str(num_encoder_layers-1))
      encoder_outputs, state_h_enc = encoder_cell(encoder_inputs)
      encoder_states = [state_h_enc]

      #----------------------encoder model-------------------------------
      encoder_model = keras.Model(inputs = e_inputs, outputs = [encoder_outputs, state_h_enc])

      #-------------------------DECODER---------------------------------------------
      decoder_state_input_h = keras.Input(shape=(self.hidden_layer_size,), name="input_decoder_h")
      decoder_hidden_state_input = Input(shape=(22,self.hidden_layer_size))
      decoder_state_inputs = [decoder_state_input_h]

      d_inputs = model.input[1] 
      d_embed_layer = model.get_layer("decoder_embedding_layer")
      decoder_inputs = d_embed_layer(d_inputs)

      #-------------------------decoder cell layers---------------------------------
      decoder_cell = model.get_layer("decoder_layer_" + str(num_decoder_layers-1))
      decoder_outputs, state_h_dec = decoder_cell(decoder_inputs, initial_state = decoder_state_inputs)
      decoder_states = [state_h_dec]

      #-----------------------------attention inference-----------------------------------
      attn_layer = model.get_layer("attention_layer")
      attn_inf_out, attn_inf_states = attn_layer([decoder_hidden_state_input, decoder_outputs])

      #------------------------------concatenation--------------------------------------
      concate = model.get_layer("concat_layer")
      decoder_inf_concat = concate([decoder_outputs, attn_inf_out])

      #----------- A dense softmax layer to get generate probability over the target vocabulary -----------
      time_dist_dense = model.get_layer("time_distributed_layer")
      decoder_inf_pred = time_dist_dense(decoder_inf_concat)


      decoder_model = keras.Model( 
                              [d_inputs] + [decoder_hidden_state_input, decoder_state_input_h],
                              [decoder_inf_pred] + [state_h_dec]
                            )
      
    reverse_input_char_index = dict((i, char) for char, i in self.input_token_index.items())
    reverse_target_char_index = dict((i, char) for char, i in self.target_token_index.items())
    return (reverse_input_char_index,reverse_target_char_index, encoder_model, decoder_model)

##################################################function for decoding input sequence ###########################################

  def decode_sequence(self, input_data, encoder_model, decoder_model, max_decoder_seq_length, target_token_index):

    num_examples = input_data.shape[0]
    max_encoder_seq_length = input_data.shape[0]

    #---------------- raw form of predictions to be returned --------------------------------
    predicted_output = np.full((num_examples, max_decoder_seq_length), target_token_index[" "])

    # ------ -----------if cell type=LSTM ---------------------------------
    if self.cell_type =="lstm":
      e_out, e_h, e_c = encoder_model.predict(input_data[:num_examples,:])
      states = [e_h, e_c]      
      target_seq = np.zeros((num_examples, 1),dtype="int32")
      target_seq[:, 0] = self.target_token_index["\t"]  

    # ------ -----------if cell type= Simple RNN or GRU --------------------
    elif self.cell_type == "rnn" or self.cell_type =="gru":
      e_out, e_h = encoder_model.predict(input_data[:num_examples,:])
      states_value = [e_h]
      target_seq = np.zeros((num_examples, 1),dtype="int32")
      target_seq[:, 0] = self.target_token_index["\t"]  

    for i in range(max_decoder_seq_length):
      # ------------------if cell type=LSTM ---------------------------------
      if self.cell_type =="lstm":
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])
        e_h, e_c = h, c
      # ------------------if cell type= Simple RNN or GRU --------------------
      elif self.cell_type =="rnn" or self.cell_type =="gru":
        output_tokens, h = decoder_model.predict([target_seq] + [e_out, e_h])
        e_h = h

      for j in range(num_examples):
        target_seq[j,0] = np.argmax(output_tokens[j, -1, :])
        if target_seq[j,0] == target_token_index["\n"]:
          predicted_output[j,i] = target_token_index[" "]
        else:
          predicted_output[j,i] = target_seq[j,0]

  #--------------------------- return predictions ------------------------------
    return predicted_output
  
  ################################################## function for calculating accuracy ###########################################

  def calculate_accuracy(self, output_corpus, input_data, reverse_target_char_index, encoder_model, decoder_model):
    limit = input_data.shape[0]   

    #------------------------------ creating vectorised form of true output ------------------------------
    true_output = np.full((limit, self.max_decoder_seq_length), self.target_token_index[" "])
    for row in range(limit):
      example = output_corpus[row].replace("\n","").replace("\t","")
      for col in range(len(example)):
        true_output[row][col] = self.target_token_index[example[col]]
    
    #------------------------------ creating vectorised form of predicted output --------------------------------
    predicted_output = self.decode_sequence(input_data, encoder_model, decoder_model, self.max_decoder_seq_length, target_token_index)

    #------------------------------ calculating accuracy ------------------------------------
    A = true_output
    B = predicted_output 
    accuracy = (np.count_nonzero((A == B).all(1))/A.shape[0])

    # wandb.log({"val_acc":accuracy})

    return (accuracy, predicted_output)

#===================================== end of class Machine_Transliterator ==========================================



**PARAMETERS**

In [ ]:
cell_type = "lstm"
batch_size = 32  # Batch size for training.
epochs = 30  # Number of epochs to train for.
hidden_layer_size= 256  # Latent dimensionality of the encoding space.
activation = "softmax"
optimizer = "Adam"
encoder_embed_size = 1024 #Encoder embedsize
decoder_embed_size = 27 #Decoder embedsize
num_encoder_layers= 1  # number of hidden layers in encoder
num_decoder_layers= 1    # number of hidden layers in decoder
dropout=0.1 #Dropout

**PREPROCESSING THE DATA**

In [ ]:
############################# preprocessing the data ################################

(encoder_input_data,decoder_input_data,decoder_target_data, _, _)=data(train_data_path,input_token_index,target_token_index)
(val_encoder_input_data,val_decoder_input_data,val_decoder_target_data, val_target_texts, val_input_texts)=data(validation_data_path ,input_token_index,target_token_index)

In [ ]:
(t_encoder_input_data,t_decoder_input_data,t_decoder_target_data, t_target_texts, t_input_texts)=data(test_data_path ,input_token_index,target_token_index)

**CREATING MACHINE TRANSLITERATOR**

In [ ]:
########################## creating machine transliterator object ###############################
machine = Machine_Transliterator(
      max_encoder_seq_length,max_decoder_seq_length,encoder_embed_size, 
      decoder_embed_size,num_encoder_layers,num_decoder_layers,
      batch_size, hidden_layer_size, num_encoder_tokens, cell_type, num_decoder_tokens, 
      input_token_index,target_token_index, activation, optimizer,dropout
      )

**TRAINING**

In [ ]:
model = machine.train_model(
    encoder_input_data, decoder_input_data,decoder_target_data,epochs,batch_size,
    val_encoder_input_data, val_decoder_input_data, val_decoder_target_data
    )

Epoch 1/30
1382/1382 [==============================] - 113s 57ms/step - loss: 0.8375 - accuracy: 0.7915 - val_loss: 0.1942 - val_accuracy: 0.9398
Epoch 2/30
1382/1382 [==============================] - 78s 57ms/step - loss: 0.1867 - accuracy: 0.9399 - val_loss: 0.1625 - val_accuracy: 0.9476
Epoch 3/30
1382/1382 [==============================] - 78s 57ms/step - loss: 0.1501 - accuracy: 0.9509 - val_loss: 0.1444 - val_accuracy: 0.9532
Epoch 4/30
1382/1382 [==============================] - 77s 56ms/step - loss: 0.1298 - accuracy: 0.9576 - val_loss: 0.1400 - val_accuracy: 0.9544
Epoch 5/30
1382/1382 [==============================] - 76s 55ms/step - loss: 0.1134 - accuracy: 0.9630 - val_loss: 0.1367 - val_accuracy: 0.9559
Epoch 6/30
1382/1382 [==============================] - 75s 55ms/step - loss: 0.1028 - accuracy: 0.9662 - val_loss: 0.1365 - val_accuracy: 0.9565
Epoch 7/30
1382/1382 [==============================] - 75s 55ms/step - loss: 0.0941 - accuracy: 0.9692 - val_loss: 0.1346 

In [ ]:
# tf.keras.utils.plot_model(model,to_file='model.png',show_shapes=True)

**INFERENCE**

In [ ]:
(reverse_input_char_index,reverse_target_char_index, encoder_model, decoder_model) = machine.sampling_inference(model, num_encoder_layers, num_decoder_layers)

**ACCURACY**

In [ ]:
################################# Validation Accuracy ###########################################

input_data = val_encoder_input_data #english vectors
output_corpus = val_target_texts #hindi words
val_acc,_ = machine.calculate_accuracy(output_corpus, input_data, reverse_target_char_index, encoder_model, decoder_model)
print(val_acc)

0.4153281321707205


In [ ]:
################################# Testing Accuracy #############################################

input_data = t_encoder_input_data #english vectors
output_corpus = t_target_texts #hindi words
test_acc,predicted_output= machine.calculate_accuracy(output_corpus, input_data, reverse_target_char_index, encoder_model, decoder_model)
print(test_acc)

0.4153709462461128


**PREPARING CSV FILE**

In [ ]:
#--------------------------------caution: terminal commands ---------------------------------------------
%cd
%cd ../content
!rm predictions.*

/root
/content
rm: cannot remove 'predictions.*': No such file or directory


In [ ]:
#---------------------------------Creating predictions for predictions_vanilla ---------------------------------------------

num_rows = predicted_output.shape[0]
num_columns = predicted_output.shape[1]

#-------------------------------- Creating CSV file ---------------------------------------------
inp_file = open("predictions.csv", "w")
col_names = "Input_text,True_output,Predicted_output,Matched,Accuracy_till_now"+"\n"
inp_file.write(col_names)

count = 0
accuracy_till_now = 0
matched = False

predictions = []

for i in range(num_rows):
  decoded_sentence = ""
  for j in range(num_columns):
    decoded_sentence += reverse_target_char_index[predicted_output[i,j]]
  decoded_sentence = "".join(decoded_sentence.split())
  predictions.append(decoded_sentence)
  if decoded_sentence == t_target_texts[i].replace("\t","").replace("\n",""):
    matched = True
    count += 1
  else:
    matched = False
  accuracy_till_now = (count/(i+1))
  line = str(t_input_texts[i].replace("\t","").replace("\n","")) + "," + str(t_target_texts[i].replace("\t","").replace("\n","")) + "," + str(decoded_sentence) + ","+ str(matched) +","+ str(accuracy_till_now)+"\n"
  inp_file.write(line)

#-------------------------------- Closing CSV file ---------------------------------------------
inp_file.close()